In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [9]:
# lineare klassifikation
def e_in(X, y, w):
    return np.sum(H(X, w) != y) / X.shape[0]

# perzeptron
def h(x, w):
    return np.sign(x @ w)

def H(X, w):
    return [h(x, w) for x in X]

# p(x)
def p(x, w):
    return -w[1]/w[2]*x - w[0]/w[2]

def P(X, w):
    return [p(x, w) for x in X]

# pla
def pla(X, y, w=None, t=0):
    if t == 0:
        X = np.column_stack((np.ones(X.shape[0]), X))
    if w is None:
        w = np.zeros(X.shape[1])

    for i, x in enumerate(X):
        if h(x, w) != y[i]:
            return pla(X, y, w + y[i]*x, t+1)

    return w

# pocket
def pocket(X, y, T):
    X = np.column_stack((np.ones(X.shape[0]), X))

    w = np.zeros(X.shape[1])
    w_d = w.copy()
    e = 1

    for _ in range(T):
        for i, x in enumerate(X):
            if h(x, w) != y[i]:
                w += y[i]*x

                if e_in(X, y, w) < e:
                    w_d = w.copy()
                    e = e_in(X, y, w_d)

    return w_d

# scatter
def scatter(X, y, w):
    xlim = (X[:,0].min()-0.1, X[:,0].max()+0.1)
    ylim = (X[:,1].min()-0.1, X[:,1].max()+0.1)

    plt.scatter(X[:,0], X[:,1], c=y)
    plt.plot(xlim, P(xlim, w))
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [ ]:
# lineare regression
def lin_reg(X, y):
    X = np.column_stack((np.ones(X.shape[0]), X))

    X_d = np.linalg.inv(X.T @ X) @ X.T
    return X_d @ y

def h(x, w):
    return np.sum(np.hstack((np.ones(1), x)) + w)

def H(X, w):
    return tuple([h(x, w) for x in X])

# scatter
def scatter(X, y, w):
    xlim = (X.min()-0.1, X.max()+0.1)
    ylim = (y.min()-0.1, y.max()+0.1)

    plt.scatter(X, y)
    plt.plot(xlim, H(xlim, w))
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()